In [ ]:
#Import the libraries necessary to run this program
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import Image, ImageTk

def open_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        file_label.config(text="Selected File: " + file_path)
        file_path_var.set(file_path)

def save_response1(root, *args):
    response1_text = response1.get()

def save_response2(root, *args):
    response2_text = response2.get()

def save_response3(root, *args):
    response3_text = response3.get()

def save_response4(root, *args):
    response4_text = response4.get()

def save_response5(root, *args):
    response5_text = response5.get()

def save_response6(root, *args):
    response6_text = response6.get()

def save_response8(root, *args):
    response8_text = response8.get()

def toggle_input_field():
    if c_v1.get() == 1:  # Checkbox is checked
        response_label5.grid(row=11, column=0, padx=10, pady=5, sticky="w")
        response5_entry.grid(row=12, column=0, padx=10, pady=5, sticky="w")
    else:
        response_label5.grid_forget()  # Hide the label field when checkbox is unchecked
        response5_entry.grid_forget()  # Hide the input field when checkbox is unchecked

def toggle_input_field2():
    if c_v2.get() == 1:  # Checkbox is checked
        response_label6.grid(row=14, column=0, padx=10, pady=5, sticky="w")
        response6_entry.grid(row=15, column=0, padx=10, pady=5, sticky="w")
    else:
        response_label6.grid_forget()  # Hide the label field when checkbox is unchecked
        response6_entry.grid_forget()  # Hide the input field when checkbox is unchecked

def toggle_input_field3():
    if c_v3.get() == 1:  # Checkbox is checked
        response_label8.grid(row=19, column=0, padx=10, pady=5, sticky="w")
        response8_entry.grid(row=20, column=0, padx=10, pady=5, sticky="w")
    else:
        response_label8.grid_forget()  # Hide the label field when checkbox is unchecked
        response8_entry.grid_forget()  # Hide the input field when checkbox is unchecked

def update_line():
    selected_option.get() in ['a', 'all']
    selected_option.get() in ['b', 'all']
    selected_option.get() == 'all'

def execute_function():
    loadpath = file_path_var.get()
    Blank_Name = response1.get()
    Blank_Num = int(response2.get())
    STD_Name = response3.get()
    STD_Num = int(response4.get())
    Sens_Inj = c_v1.get()
    SensSTD = response5.get()
    Chk_Inj = c_v2.get()
    STD2 = response6.get()
    Reps = selected_option.get()
    Excip_Inj = c_v3.get()
    Excip = response8.get()
    path = save_path_var.get()

    # Perform all RRT Table calculations and export table after hitting execute button

    # Import the file and save it as a Data Frame
    DF= pd.read_csv(loadpath, sep="\t")

    #Create a new list for Injection Ids
    Inj_Ids=[]
    Inj_Ids=DF['Injection Id'].unique()

    #Create a Data Frame for Impurity Analysis
    Impurity_DF=pd.DataFrame()
    Impurity_DF['Sample_Name']=DF['SampleName']
    Impurity_DF['Injection_Ids']=DF['Injection Id']
    Impurity_DF['Retention_Time']=DF['Retention Time']
    Impurity_DF['Area']=DF['Area']

    # Save lists with the number of blank injections and the Injection Ids
    Blank_Injs=Impurity_DF[Impurity_DF['Sample_Name']==Blank_Name].Injection_Ids.unique().tolist()
    Num_Blank_Injs=len(Impurity_DF[Impurity_DF['Sample_Name']==Blank_Name].Injection_Ids.unique().tolist())

    # Save the Injection ID of the analysis standard
    Anal_Blank=Blank_Injs[Blank_Num-1]

    # Remove the Injection ID of the analysis standard from the list of Standard Injection Ids
    Blank_Injs.remove(Anal_Blank)

    # Storing the Standard index values of the Impurity Data Frame to drop
    Blank_Index=[]
    for i in Blank_Injs:
      Blank_Index.append(Impurity_DF[Impurity_DF['Injection_Ids']==i].index.values)

    #Removing the unwanted standard injections from the Impurity Data Frame
    for i in Blank_Index:
      Impurity_DF=Impurity_DF.drop(i)

    # Create a new Data Frame for blanks peaks
    Blank_DF=pd.DataFrame()
    # List all peak retention times and areas to add to DF
    Blank_DF['Area']=Impurity_DF[Impurity_DF['Injection_Ids']==Anal_Blank].Area.reset_index(drop=True)
    Blank_DF['Blank_RT']=Impurity_DF[Impurity_DF['Injection_Ids']==Anal_Blank].Retention_Time.reset_index(drop=True)

    # Remove the blank from the Impurity Data Frame since it has been saved in the Blank Data Frame
    Impurity_DF.drop(Impurity_DF[Impurity_DF['Sample_Name']==Blank_Name].index, inplace=True)

    # Drop rows in the impurity data frame that don't have a peak, i.e. total impurities rows generated from Empower
    Impurity_DF.dropna(inplace= True)

    # Making sure the index of the Impurity Data Frame starts from 0
    Impurity_DF.reset_index(drop=True, inplace=True)

    # Identifying the injection sequence and creating a list of injection IDs and Sample Names
    Seq=Impurity_DF.groupby('Injection_Ids')['Sample_Name'].unique().to_frame()
    # Remove the blank from the Sequence Data Frame since
    Seq.drop(Seq[Seq['Sample_Name']==Blank_Name].index, inplace=True)

    # Create a list of the areas of the API peaks
    API_Peak_Areas=Impurity_DF.groupby('Injection_Ids')['Area'].max().to_list()

    # Identify the any injections in the API Peak Area lists that don't have a peak area
    blank_values = np.isnan(API_Peak_Areas)
    # Remove the blank values
    API_Peak_Areas = np.delete(API_Peak_Areas, blank_values)

    # Create a list for the retention time of the API peaks
    API_RT=[]

    # Loop through the Impurity Data Frame to find the API retention times and Injection IDs and add to the lists
    for a in API_Peak_Areas:
      API_RT.append(float(Impurity_DF[Impurity_DF['Area']==a].Retention_Time))

    # Create a new Table for only the API peaks that includes Injecction ID, Retention Time, and Area
    Seq['API_RT']=API_RT
    Seq['API_Peak_Areas']=(API_Peak_Areas)
    Seq.sort_values('Sample_Name', inplace=True)

    # Save lists with the number of Standard injections and the Injection Ids
    STD1_Injs=Impurity_DF[Impurity_DF['Sample_Name']==STD_Name].Injection_Ids.unique().tolist()
    Num_STD1_Injs=len(Impurity_DF[Impurity_DF['Sample_Name']==STD_Name].Injection_Ids.unique().tolist())

    # Save the Injection ID of the analysis standard
    Anal_STD=STD1_Injs[STD_Num-1]

    # Remove the Injection ID of the analysis standard from the list of Standard Injection Ids
    STD1_Injs.remove(Anal_STD)

    # Storing the Standard index values of the Impurity Data Frame to drop
    Standard_Index=[]
    for i in STD1_Injs:
      Standard_Index.append(Impurity_DF[Impurity_DF['Injection_Ids']==i].index.values)

    #Removing the unwanted standard injections from the Impurity Data Frame
    for i in Standard_Index:
      Impurity_DF=Impurity_DF.drop(i)

    # If Sensitivity and Check STD are part of sequence, drop based on their name
    if Sens_Inj==1:
      # Dropping SensSTD from the Impurity Data Frame
      Impurity_DF.drop(Impurity_DF[Impurity_DF['Sample_Name']==SensSTD].index, inplace=True)

    if Chk_Inj==1:
      # Dropping Chk STD from Impurity Data Frame
      Impurity_DF.drop(Impurity_DF[Impurity_DF['Sample_Name']==STD2].index, inplace=True)

    # Making sure the index of the Impurity Data Frame starts from 0
    Impurity_DF.reset_index(drop=True, inplace=True)

    # Select which sample replicates to include in the RRT table
    SNL=Impurity_DF.Sample_Name.to_list()
    if Reps=='a':
      for i in np.arange(len(SNL)):
        if 'b' in SNL[i]:
          Impurity_DF.drop([i], axis=0, inplace=True)
    elif Reps=='b':
      for i in np.arange(len(SNL)):
        if 'a' in SNL[i]:
          Impurity_DF.drop([i], axis=0, inplace=True)

    # Create a list of the areas of the API peaks
    API_Peak_Areas=Impurity_DF.groupby('Injection_Ids')['Area'].max().to_list()

    # Create a list for the rention time of the API peaks
    API_RT=[]

    # Create a list for the Injection ID of the API peaks
    A_Id=[]

    # Create a list for the Samples Names of each Injection ID
    A_Sampnames=[]

    # Loop through the Impurity Data Frame to find the API rention times and Injection IDs and add to the lists
    for a in API_Peak_Areas:
      A_Sampnames.append(Impurity_DF[Impurity_DF['Area']==a].Sample_Name.values)
      API_RT.append(float(Impurity_DF[Impurity_DF['Area']==a].Retention_Time))
      A_Id.append(int(Impurity_DF[Impurity_DF['Area']==a].Injection_Ids))

    # Create a new Table for only the API peaks that includes Injecction ID, Retention Time, and Area
    API_Table=pd.DataFrame()
    API_Table['A_Id']=(A_Id)
    API_Table['Sample_Name']=A_Sampnames
    API_Table['API_RT']=API_RT
    API_Table['API_Peak_Areas']=(API_Peak_Areas)

    # Make a list of every retention time for each injection ID
    RT_DF=Impurity_DF.groupby('Injection_Ids')['Retention_Time'].unique().to_frame()

    # If excipient or polymer blank was part of injection sequence.
    if Excip_Inj==1:
      # Add the excipient retention time and peak area to the Blank Data Frame
      Blank_DF.loc[Blank_DF.shape[0]] = [int(Impurity_DF[Impurity_DF['Sample_Name']==Excip].Area), float(Impurity_DF[Impurity_DF['Sample_Name']==Excip].Retention_Time)]
      # Dropping Excipient from the Impurity Data Frame
      Impurity_DF.drop(Impurity_DF[Impurity_DF['Sample_Name']==Excip].index, inplace=True)

    # Calculate RRT in the Blank Table by dividing by the Standard Retention Time
    Blank_DF['Blank_RRT']=round(Blank_DF['Blank_RT']/API_RT[0],2)

    # Drop peaks by RRT if blank peak is also present in excipient blank to avoid double subtraction
    Blank_DF = Blank_DF.drop_duplicates(subset=['Blank_RRT'], keep='first')

    # Make a list to save calulated RRTs
    RRT = []

    # For each retention time calculate the RRT and save it to the list
    for i in range(len(API_Table.A_Id)):
      Cur_ID = int(API_Table.A_Id[i])
      Cur_RT = API_Table.API_RT[i]
      if Cur_ID in RT_DF.index:
        for j in range(len(RT_DF.loc[Cur_ID])):
          Inj = RT_DF.loc[Cur_ID]
          RRT.append(np.round((Inj[j] / Cur_RT),3))
        else:
          continue

    # Defining a function to flatten a two dimensional list into a single list
    def flatten_array(array_of_arrays):
        return [item for sublist in array_of_arrays for item in sublist]

    # Saving a list of RRTs in 1D that was flattened from 2D.
    flat_RRT = flatten_array(RRT)

    # Add the list of RRTs as a new column to the Impurity Data Frame
    Impurity_DF['RRT']=flat_RRT

    # Making sure the index of the Impurity Data Frame starts from 0
    Impurity_DF.reset_index(drop=True, inplace=True)

    # Round the RRT values to 2 places
    Impurity_DF['RRT']=round(Impurity_DF['RRT'],2)

    # Remove rows that don't have a peak from the blank Data Frame
    Blank_DF.dropna(inplace= True)
    # Reset the Blank Data Frame index
    Blank_DF.reset_index(drop=True, inplace=True)

    # Looping through all blank peaks and subtracting them from peaks in the Impurity Data Frame if the RRTs match
    Loop_pos=0
    Area_Blank_Sub=Impurity_DF.Area.reset_index(drop=True)

    for i in Blank_DF['Blank_RRT']:
      if (Impurity_DF.RRT==(i)).any():
        for j in np.arange(len(Area_Blank_Sub)):
          if Impurity_DF.RRT.reset_index(drop=True)[j]==i:
            Area_Blank_Sub[j]=(Area_Blank_Sub[j]-Blank_DF.Area[Loop_pos])
            if Area_Blank_Sub[j]<0:                                           # If subtracting the blank peak results in a negative number, set the area equal to 0
              Area_Blank_Sub[j]=np.nan
            else:
              continue
          else:
            continue
        Loop_pos+=1
      else:
        Loop_pos+=1
        continue

    # Resetting the Area Column to be the Blank Subtracted Adjusted Area
    Impurity_DF['Area']=Area_Blank_Sub

    # For every injection calculate the cumulative area and make sure its greater than zero, drop injections with no peaks from dataframe
    for i in range(len(API_Table.A_Id)):
      Cur_ID = int(API_Table.A_Id[i])
      Cum_Area = Impurity_DF.groupby('Injection_Ids')['Area'].sum()
      Cur_Cum_Area= int(Cum_Area[Cur_ID])
      if Cur_Cum_Area == 0:
            Impurity_DF.drop(Impurity_DF[Impurity_DF['Injection_Ids']==Cur_ID].index, inplace=True)
            API_Table.drop(API_Table[API_Table['A_Id']==Cur_ID].index, inplace=True)
    # Making sure the index of the Data Frames start from 0
    Impurity_DF.reset_index(drop=True, inplace=True)
    API_Table.reset_index(drop=True, inplace=True)

    # Make a list to store calculated percent areas
    Per_Area = []

    # For every peak calculate the percent area by dividing by the cumulative area of that Injection ID
    for i in range(len(API_Table.A_Id)):
      Cur_ID = int(API_Table.A_Id[i])
      Cum_Area = Impurity_DF.groupby('Injection_Ids')['Area'].sum()
      Cur_Cum_Area= int(Cum_Area[Cur_ID])
      if Cur_ID in Impurity_DF.Injection_Ids.to_list():
        for j in range(len(Impurity_DF[Impurity_DF['Injection_Ids']==Cur_ID].Area)):
          Areas = Impurity_DF[Impurity_DF['Injection_Ids']==Cur_ID].Area.to_list()
          Per_Area.append(Areas[j]*100/Cur_Cum_Area)
          next
        else:
          continue

    # Add a Percent Area column to the Impurity DF
    Impurity_DF['Percent_Area']=Per_Area

    # Add a new column to the Impurity DF that calculates if the percent area is above 0.05%
    Impurity_DF['Above_LOQ']=Impurity_DF.Percent_Area>=0.05

    # Add a new column to the Impurity DF that includes on peak areas above the 0.05% LOQ threshold
    Impurity_DF['Area_Above_LOQ']=Impurity_DF.Above_LOQ*Impurity_DF.Area

    # Recalculate percent areas of only peaks that are above LOQ
    # Make a list to store calculated percent areas
    Per_Area = []

    # For every peak calculate the percent area by dividing by the cumulative area of Peaks>LOQ of that Injection ID
    for i in range(len(API_Table.A_Id)):
      Cur_ID = int(API_Table.A_Id[i])
      Cum_Area = Impurity_DF.groupby('Injection_Ids')['Area_Above_LOQ'].sum()
      Cur_Cum__Area= int(Cum_Area[Cur_ID])
      if Cur_ID in Impurity_DF.Injection_Ids.to_list():
        for j in range(len(Impurity_DF[Impurity_DF['Injection_Ids']==Cur_ID].Area)):
          Areas = Impurity_DF[Impurity_DF['Injection_Ids']==Cur_ID].Area.to_list()
          Per_Area.append(Areas[j]*100/Cur_Cum__Area)
          next
        else:
          continue

    # Add a new column to the Impurity Data Frame that is the Reportable Percent Areas
    Impurity_DF['Percent_Area']=Per_Area

    # Remove RRT 1.0 from the Impurity Data Frame
    Impurity_DF.drop(Impurity_DF[Impurity_DF['RRT']==1.00].index, inplace=True)

    # Construct an RRRT table of percent areas by RRT according to sample name with a pivot table
    pivottable = pd.pivot_table(Impurity_DF, index='RRT', columns=['Sample_Name'], values='Percent_Area', aggfunc=lambda x: x) # Using lambda to pass multiple values when there are two peaks with same RRT in one sample

    # Create a function to combine sparse RRT rows if the RRT is +/- 0.01 and the rows can be considered the same peak
    def RRT_Consolidator(DF):
      DF_Shape=DF.shape
      DF_Cols=DF_Shape[1]
      DF_Rows=DF_Shape[0]
      for row in DF.index:
        if DF[DF.index==row].isnull().sum().sum()>=0.5*DF_Cols:
          if (DF.index==(row-0.01)).any():
            if DF[DF.index==row].isnull().sum().sum()>DF[DF.index==(row-0.01)].isnull().sum().sum():
              #Check if cells are empty and move to more populated row.
              Loop_pos=0
              count=0
              row_array=DF[DF.index==row].values[0]
              for val in row_array:
                if np.isnan(val):
                  Loop_pos+=1
                  continue
                else:
                  #Move cell to new row as long as the new cell is empty.
                  if np.isnan(DF[DF.index==(row-0.01)].values[0,Loop_pos]):
                    new_row=DF[DF.index==(row-0.01)].values.tolist()[0]
                    new_row[Loop_pos]=val # Add the value in the correct column to the new row
                    DF.loc[DF.index==(row-0.01)]=new_row
                    row_array[Loop_pos]=np.nan # Remove the value from the correct column in the old row
                    DF[DF.index==row]=row_array # Save the updated row to the data frame
                    DF=DF.sort_index()
                    Loop_pos+=1
                    count+=1
                    if DF_Cols==DF[DF.index==row].isnull().sum().sum():
                      #if all the items in the row have been moved, delete the row.
                      DF=DF.drop(index=row)
                    else:
                      continue
                  else:
                    Loop_pos+=1
                    continue

            elif DF[DF.index==row].isnull().sum().sum()==DF[DF.index==(row-0.01)].isnull().sum().sum():
              #Check if cells are empty and move to more lower RRT row.
              Loop_pos=0
              count=0
              row_array=DF[DF.index==row].values[0]
              for val in row_array:
                if np.isnan(val):
                  Loop_pos+=1
                  continue
                else:
                  #Move cell to new row as long as the new cell is empty.
                  if np.isnan(DF[DF.index==(row-0.01)].values[0,Loop_pos]):
                    new_row=DF[DF.index==(row-0.01)].values.tolist()[0]
                    new_row[Loop_pos]=val # Add the value in the correct column to the new row
                    DF.loc[DF.index==(row-0.01)]=new_row
                    row_array[Loop_pos]=np.nan # Remove the value from the correct column in the old row
                    DF[DF.index==row]=row_array # Save the updated row to the data frame
                    DF=DF.sort_index()
                    Loop_pos+=1
                    count+=1
                    if DF_Cols==DF[DF.index==row].isnull().sum().sum():
                      #if all the items in the row have been moved, delete the row.
                      DF=DF.drop(index=row)
            else:
                continue
          elif (DF.index==(row+0.01)).any():
            if DF[DF.index==row].isnull().sum().sum()>DF[DF.index==(row+0.01)].isnull().sum().sum():
              #Check if cells are empty and move to more populated row.
              Loop_pos=0
              count=0
              row_array=DF[DF.index==row].values[0]
              for val in row_array:
                if np.isnan(val):
                  Loop_pos+=1
                  continue
                else:
                  #Move cell to new row as long as the new cell is empty.
                  if np.isnan(DF[DF.index==(row+0.01)].values[0,Loop_pos]):
                    new_row=DF[DF.index==(row+0.01)].values.tolist()[0]
                    new_row[Loop_pos]=val # Add the value in the correct column to the new row
                    DF.loc[DF.index==(row+0.01)]=new_row
                    row_array[Loop_pos]=np.nan # Remove the value from the correct column in the old row
                    DF[DF.index==row]=row_array # Save the updated row to the data frame
                    DF=DF.sort_index()
                    Loop_pos+=1
                    count+=1
                    if DF_Cols==DF[DF.index==row].isnull().sum().sum():
                      #if all the items in the row have been moved, delete the row.
                      DF=DF.drop(index=row)
                    else:
                      continue
                  else:
                    Loop_pos+=1
                    continue
            else:
              continue
          else:
            continue
        else:
          continue
      return DF

    # Run the consolidator function and save the table result
    pivottable=RRT_Consolidator(pivottable)

    # Function to replace values less than 0.05 with "<0.05%"
    def replace_values(val):
        new_values = []
        if type(val) == np.ndarray:
            for v in val:
                if float(v) < 0.05:
                    new_values.append("<0.05%")
                else:
                   new_values.append(v)
        else:
          if val < 0.05:
              new_values="<0.05%"
          else:
              new_values=val

        return new_values

    # Apply the function to the entire DataFrame
    pivottable = pivottable.applymap(replace_values)

    # Define a function to convert values to numeric (ignores non-numeric strings)
    def convert_to_numeric(value):
        try:
            return pd.to_numeric(value)
        except:
            return np.nan

    # Apply the conversion function to the entire DataFrame
    numeric_df = pivottable.applymap(convert_to_numeric)

    # Define a custom function to calculate the sum of elements in a cell
    def custom_sum(cell_value):
        if isinstance(cell_value, np.ndarray):
            return sum(cell_value)
        else:
            return cell_value

    # Apply the custom_sum function with a loop to the specified column and add to a list
    cumulative_sum=[]
    for col in numeric_df.columns:
      cumulative_sum.append(round(numeric_df[col].apply(custom_sum).sum(),2))

    # Adding a Total Impurities row at the bottom of the pivot table
    pivottable.loc['Total Impurities']=cumulative_sum

    # Function to round numeric values in data frame
    def round_values(val):
        new_values = []
        if type(val) == list:
            for v in val:
              if type(v)== np.float64:
                new_values.append(round(v,2))
              else:
                new_values.append(v)
            return new_values
        elif type(val) == str:
            return val
        elif type(val) == np.float64:
            round_value=round(val,2)
            return round_value
        elif type(val) == float:
            round_value=round(val,2)
            return round_value
        else:
          return val

    # Apply the function to the entire DataFrame
    pivottable = pivottable.applymap(round_values)

    with pd.ExcelWriter(path) as engine:
        pivottable.to_excel(excel_writer=engine, sheet_name='RRT_Table', index=True)
        Seq.to_excel(excel_writer=engine, sheet_name='API_Table')

    # Update the status label to indicate successful execution
    status_label.config(text="App ran successfully!")

def browse_folder():
    folder_path = filedialog.askdirectory()
    if folder_path:
        selected_folder_label.config(text=f"Selected Folder: {folder_path}")
        global selected_folder_path
        selected_folder_path = folder_path

def save_file():
    if selected_folder_path:
        save_path = filedialog.asksaveasfilename(defaultextension=".xlsx")
        if save_path:
            saved_label.config(text=f"Filepath saved to: {save_path}")
            save_path_var.set(save_path)

# Create the main window
root = tk.Tk()
root.title("RRT Table Generator")

root.geometry("750x900")  # Size of the window
root.geometry('+%d+%d'%(350,10)) #place GUI at x=350, y=10

#logo
#resizing the displayed image while keeping its ratio
def resize_image(img):
    width, height = int(img.size[0]), int(img.size[1])
    height = int(150/width*height)
    width = 150
    img = img.resize((width, height))
    return img

logo = Image.open('RRT Icon.png')
logo = resize_image(logo)
logo = ImageTk.PhotoImage(logo)
logo_label = tk.Label(image=logo)
logo_label.image = logo
logo_label.grid(column=2, row=0)

# Create variables to store the responses
file_path_var = tk.StringVar()

response1 = tk.StringVar()

response2 = tk.IntVar(value=0)

response3 = tk.StringVar()

response4 = tk.IntVar(value=0)

response5 = tk.StringVar()

response6 = tk.StringVar()

response8 = tk.StringVar()

c_v1=IntVar()

c_v2=IntVar()

c_v3=IntVar()

selected_filepath = ""
save_path_var = tk.StringVar()

# Create a radio button variable
selected_option = tk.StringVar()

# Initialize the selected_option variable
selected_option.set(None)

# Create widgets
open_button = tk.Button(root, text="Open File", command=open_file)
file_label = tk.Label(root, text="Selected File: None")

# Questions and Response
response_label1 = tk.Label(root, text="Please input how the blank is named, must be exact. e.g. Blank")
response1_entry = tk.Entry(root, textvariable=response1)
# Bind the response variable to a callback
response1.trace_add('write', save_response1)

# Questions and Response
response_label2 = tk.Label(root, text="Please input which blank to use, input number e.g. 3")
response2_entry = tk.Entry(root, textvariable=response2)
# Bind the response variable to a callback
response2.trace_add('write', save_response2)

# Questions and Response
response_label3 = tk.Label(root, text="Please input how the standard you want to use for impurity analysis is named, must be exact. e.g. STD1")
response3_entry = tk.Entry(root, textvariable=response3)
# Bind the response variable to a callback
response3.trace_add('write', save_response3)

# Questions and Response
response_label4 = tk.Label(root, text="Please input which standard injection to use, input number e.g. 1 or 5")
response4_entry = tk.Entry(root, textvariable=response4)
# Bind the response variable to a callback
response4.trace_add('write', save_response4)

# Questions and Response
response_label5 = tk.Label(root, text="Please input how the Sensitivity Standard is named, must be exact. e.g. SensSTD")
response5_entry = tk.Entry(root, textvariable=response5)
# Bind the response variable to a callback
response5.trace_add('write', save_response5)

# Questions and Response
response_label6 = tk.Label(root, text="Please input how the Check Standard is named, must be exact. e.g. STD2")
response6_entry = tk.Entry(root, textvariable=response6)
# Bind the response variable to a callback
response6.trace_add('write', save_response6)

check = tk.Checkbutton(root,text='Sample set includes Sensitivity Standard',variable=c_v1, onvalue=1, offvalue=0,
	command=toggle_input_field)

check2 = tk.Checkbutton(root,text='Sample set includes Check Standard',variable=c_v2,
	onvalue=1,offvalue=0,command=toggle_input_field2)

response_label7 = tk.Label(root, text="What sample replicates to include in the RRT Table? (a, b, all)")

# Create radio buttons
radio_button_a = tk.Radiobutton(root, text="Replicate A", variable=selected_option, value='a', command=update_line)
radio_button_b = tk.Radiobutton(root, text="Replicate B", variable=selected_option, value='b', command=update_line)
radio_button_all = tk.Radiobutton(root, text="All Replicates", variable=selected_option, value='all', command=update_line)

check3 = tk.Checkbutton(root,text='Sample set includes a polymer or excipient blank', variable=c_v3,
	onvalue=1,offvalue=0,command=toggle_input_field3)

# Questions and Response
response_label8 = tk.Label(root, text="Please input how the polymer or excipient blank is named, must be exact. e.g. SSF")
response8_entry = tk.Entry(root, textvariable=response8)
# Bind the response variable to a callback
response8.trace_add('write', save_response8)

# Create the execute button to complete the program
execute_button = tk.Button(root, text="Execute", command=execute_function)

# Browse folders and save the file
browse_button = tk.Button(root, text="Browse Folder", command=browse_folder)

selected_folder_label = tk.Label(root, text="Select Save Folder: ")

save_button = tk.Button(root, text="Save Filename", command=save_file)

saved_label = tk.Label(root, text="")

status_label = tk.Label(root, text="")

# Place widgets using grid layout
open_button.grid(row=1, column=0, padx=10, pady=3, sticky="w")
file_label.grid(row=1, column=0, padx=10, pady=3, columnspan=3, sticky="e")

response_label1.grid(row=2, column=0, padx=10, pady=3, columnspan=2, sticky="w")
response1_entry.grid(row=3, column=0, padx=10, pady=3, columnspan=3, sticky="w")

response_label2.grid(row=4, column=0, padx=10, pady=3, columnspan=2, sticky="w")
response2_entry.grid(row=5, column=0, padx=10, pady=3, columnspan=3, sticky="w")

response_label3.grid(row=6, column=0, padx=10, pady=3, columnspan=2, sticky="w")
response3_entry.grid(row=7, column=0, padx=10, pady=3, columnspan=3, sticky="w")

response_label4.grid(row=8, column=0, padx=10, pady=3, columnspan=2, sticky="w")
response4_entry.grid(row=9, column=0, padx=10, pady=3, columnspan=3, sticky="w")

check.grid(row=10,column=0, padx=10, pady=10, sticky="w")

response_label5.grid(row=11, column=0, padx=10, pady=3, columnspan=2, sticky="w")
response5_entry.grid(row=12, column=0, padx=10, pady=3, columnspan=3, sticky="w")
response_label5.grid_forget()  # Hide the label field when checkbox is unchecked
response5_entry.grid_forget()  # Hide the input field when checkbox is unchecked

check2.grid(row=13,column=0, padx=10, pady=10, sticky="w")

response_label6.grid(row=14, column=0, padx=10, pady=3, columnspan=2, sticky="w")
response6_entry.grid(row=15, column=0, padx=10, pady=3, columnspan=3, sticky="w")
response_label6.grid_forget()  # Hide the label field when checkbox is unchecked
response6_entry.grid_forget()  # Hide the input field when checkbox is unchecked

response_label7.grid(row=16, column=0, padx=10, pady=10, columnspan=2, sticky="w")

radio_button_a.grid(row=17, column=0, columnspan=1, padx=10, pady=3, sticky="e")
radio_button_b.grid(row=17, column=1, columnspan=1, padx=10, pady=3, sticky="w")
radio_button_all.grid(row=17, column=2, columnspan=1, padx=10, pady=3, sticky="w")

check3.grid(row=18,column=0, padx=10, pady=10, sticky="w")

response_label8.grid(row=19, column=0, padx=10, pady=3, columnspan=2, sticky="w")
response8_entry.grid(row=20, column=0, padx=10, pady=3, columnspan=3, sticky="w")
response_label8.grid_forget()  # Hide the label field when checkbox is unchecked
response8_entry.grid_forget()  # Hide the input field when checkbox is unchecked

browse_button.grid(row=21, column=0, columnspan=1, padx=10, pady=3, sticky="w")
selected_folder_label.grid(row=22, column=0, columnspan=3, padx=3, pady=10, sticky="w")

save_button.grid(row=23, column=0, columnspan=1, padx=10, pady=3, sticky="w")
saved_label.grid(row=24, column=0, columnspan=3, padx=10, pady=3, sticky="w")

execute_button.grid(row=25, column=0, columnspan=1, padx=10, pady=3, sticky="w")

status_label.grid(row=25, column=1, columnspan=1, padx=10, pady=3, sticky="w")

# Tkinter event loop
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\thurman.falk\AppData\Local\anaconda3\Lib\tkinter\__init__.py", line 592, in get
    return self._tk.getint(value)
           ^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: expected integer but got ""

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\thurman.falk\AppData\Local\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\thurman.falk\AppData\Local\Temp\ipykernel_20892\1389806722.py", line 22, in save_response2
    response2_text = response2.get()
                     ^^^^^^^^^^^^^^^
  File "C:\Users\thurman.falk\AppData\Local\anaconda3\Lib\tkinter\__init__.py", line 594, in get
    return int(self._tk.getdouble(value))
               ^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: expected floating-point number but got ""
Exception in Tkinter callback
Traceb